## Exercise 1

### Imports

In [1]:
import os.path
from pyspark import Broadcast
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, ArrayType
from itertools import combinations
from typing import Iterable, Any, List

### Spark initialization

In [2]:
spark = SparkSession.builder \
    .appName('exercise1') \
    .config('spark.master', 'local[*]') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/05 16:55:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Prepare the data

In [14]:
tracks_df = (spark.read
    .csv('data/tracks.csv.gz')
)
# rename columns with row values from first row to second row
tmp = tracks_df.take(2)
columns = tracks_df.columns
for i in range(1,len(tmp[0])):
    tracks_df = tracks_df.withColumnRenamed(columns[i], str(tmp[0][i]) + "-" + str(tmp[1][i]))

tracks_df = (tracks_df.withColumnRenamed(columns[0], "track_id")
    .filter(col("track_id").isNotNull()) 
    .filter(col("track_id") != "track_id")
    .filter(col("set-subset") == "small")
)


tracks_df.show()

+--------+--------------+-------------------+-------------------+-----------------+---------------+--------+--------------------+-------------+--------------+-------------+--------------------+------------+----------------+------------------------+----------------------+------------------------+--------------------+---------------+-------------------+----------------+---------+---------------+----------------+----------------+--------------------+--------------------+-----------------------+--------------------+--------------------+---------------------+----------+----------+--------------+--------------+--------------+-------------------+-------------------+--------------+---------------+---------------+------------+----------------+--------------------+--------------+-------------------+--------------------+-------------+--------------+------------+---------------+-------------+--------------------+
|track_id|album-comments| album-date_created|album-date_released|   album-engineer|al

In [10]:
features_df = (spark.read
    .csv('data/features.csv.gz')
)
# rename columns with row values from first row to second row
tmp = features_df.take(3)
columns = features_df.columns
for i in range(1,len(tmp[0])):
    features_df = features_df.withColumnRenamed(columns[i], str(tmp[0][i]) + "-" + str(tmp[1][i]) + "-" + str(tmp[2][i]))

features_df = (features_df.withColumnRenamed(columns[0], "track_id")
    .filter(col("track_id") != "feature")
    .filter(col("track_id") != "statistics")
    .filter(col("track_id") != "number")
    .filter(col("track_id") != "track_id")
)

features_df.show(5)

+--------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+--------------